In [ ]:
%cd /without_history

In [ ]:
num=100 # 100 or 1500

In [ ]:
import json
with open(f'./Movies/select_{num}_Movies.json', "r") as input_file:
    select_dict = json.load(input_file)

In [ ]:
data = list(select_dict.keys())

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('YOUR_MODEL_PATH', add_bos_token=False)
def get_hash(x):
    x = [str(_) for _ in x]
    return '-'.join(x)
try:
    end_of_sentence_token_id = tokenizer.encode(
        "A sentence\n", add_special_tokens=False
    )[-1]
except:
    end_of_sentence_token_id = tokenizer.convert_tokens_to_ids("\n")

In [ ]:
# qwen2.5
import numpy as np
from collections import defaultdict
def nested_defaultdict():
    return defaultdict(float)
token_distri = defaultdict(nested_defaultdict)
for d in data:
    for i in range(len(tokenizer.encode(d))):
        if i==0:
            token_distri['first'][tokenizer.encode(d)[i]] += select_dict[d]
            if i==len(tokenizer.encode(d))-1:
                token_distri[str(tokenizer.encode(d)[i])][end_of_sentence_token_id] += select_dict[d]
        elif i==len(tokenizer.encode(d))-1:
            token_distri[get_hash(tokenizer.encode(d)[:i])][tokenizer.encode(d)[i]] += select_dict[d]
            token_distri[get_hash(tokenizer.encode(d))][end_of_sentence_token_id] += select_dict[d]
        else:
            token_distri[get_hash(tokenizer.encode(d)[:i])][tokenizer.encode(d)[i]] += select_dict[d]
token = {}
for k,v in token_distri.items():
    total_sum = sum(v.values())
    v = {key: np.log(value / total_sum) for key, value in v.items()}
    token[k] = v

In [ ]:
import json
with open(f'./Movies/Movies_{num}_token_distri_qwen2.5.json', 'w') as f:
    json.dump(token, f, indent=4)

In [ ]:
with open(f'./Movies/select_{num}_Movies.json', "r") as input_file:
    top_n_prob = json.load(input_file)
token_inter = {}
for k,v in top_n_prob.items():
    for i in tokenizer.encode(k):
        if i not in token_inter:
            token_inter[i] = 0
        token_inter[i] += v
token_inter = dict(sorted(token_inter.items(), key=lambda item: item[1], reverse=True))
with open(f'./Movies/select_{num}_Movies_token.json', 'w') as f:
    json.dump(token_inter, f, indent=4)

In [ ]:
# DPO

In [ ]:
import random
def select_random_key_with_lower_value(d, a):
    reference_value = d[a]
    lower_keys = [key for key, value in d.items() if value < reference_value and key != a]
    selected_key = random.choice(lower_keys)
    return selected_key

In [ ]:
import json
size = {100:6400, 1500:12800}
n = size[num]
p = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Please recommend a movie to the user. Directly output the title of the movie.
### Response:
"""
prompt = [p for _ in range(n)]
with open(f'./Movies/select_{num}_Movies.json', "r") as input_file:
    top_100_prob = json.load(input_file)
_, min_value = min(top_100_prob.items(), key=lambda item: item[1])
chosen = [random.choice([x for x in list(top_100_prob.keys()) if top_100_prob[x] != min_value]) for _ in range(n)]

rejected = []
for i in range(n):
    rejected.append(select_random_key_with_lower_value(top_100_prob, chosen[i]))
dataset = []
for i in range(n):
    dataset.append({'prompt':prompt[i], 'rejected':rejected[i], 'chosen':chosen[i]})
with open(f'./base_line/data/dpo/Movies_{num}_dpo_train_dataset.json', 'w') as f:
    json.dump(dataset, f, indent=4)

In [ ]:
# evaluation

In [ ]:
import scipy.stats
import numpy as np
import json
import matplotlib.pyplot as plt

with open(f'./Movies/select_{num}_Movies.json', "r") as input_file:
    top_100_prob = json.load(input_file)

with open(f'RESULT_PATH.json', 'r', encoding='utf-8') as file:
    result_prob = json.load(file)

float_info = np.finfo(np.float64)
def JS_divergence(p,q):
    M=(p+q)/2
    return 0.5*scipy.stats.entropy(p, M)+0.5*scipy.stats.entropy(q, M)

def kl_divergence_discrete(before, data):
    temp = {}
    for k,v in before.items():
        if k in data:
            temp[k] = v
        else:
            print(k)
            if 'Not_in_dataset' in temp:
                temp['Not_in_dataset'] += 1
            else:
                temp['Not_in_dataset'] = 1
    before = temp
    data['Not_in_dataset'] = float_info.eps
    keys = set(before.keys()).union(data.keys())
    keys.remove('Not_in_dataset')
    print(len(keys))
    b_counts = [before.get(key,float_info.eps) for key in keys]
    c_counts = [data.get(key, float_info.eps) for key in keys]
    b_total = sum(b_counts)
    c_total = sum(c_counts)
    b_dist = [count / b_total for count in b_counts]
    c_dist = [count / c_total for count in c_counts]
    combined = sorted(zip(c_dist, b_dist), key=lambda x: x[0], reverse=True)
    sorted_a, sorted_b = zip(*combined)
    c_d = list(sorted_a)
    b_d = list(sorted_b)
    b = np.asarray(list(b_dist), dtype=np.float64)
    m = np.asarray(list(c_dist), dtype=np.float64)

    plt.figure()
    plt.bar(list(keys), b_d, label='Result Distribution', alpha=0.5)
    plt.bar(list(keys), c_d, label='Target Distribution', alpha=0.5)
    plt.legend()
    plt.title('Comparison of Two Distributions')
    plt.xlabel('Categories')
    plt.ylabel('Values')
    plt.show()

    return np.sum(m * np.log(m / b)), np.sum(b * np.log(b / m)), JS_divergence(b,m)
            
kl_mb, kl_bm, js_bm = kl_divergence_discrete(result_prob, top_100_prob)
print(kl_mb)
print(kl_bm)
print(js_bm)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import json
with open(f'./Movies/select_{num}_Movies_token.json', "r") as input_file:
    top_100_prob = json.load(input_file)
with open(f'RESULT_PATH.json', 'r', encoding='utf-8') as file:
    result_prob = json.load(file)
token_inter = {}
for k,v in result_prob.items():
    for i in tokenizer.encode(k):
        if i not in token_inter:
            token_inter[str(i)] = 0
        token_inter[str(i)] += v
token_inter = dict(sorted(token_inter.items(), key=lambda item: item[1], reverse=True))

float_info = np.finfo(np.float64)
def JS_divergence(p,q):
    M=(p+q)/2
    return 0.5*scipy.stats.entropy(p, M)+0.5*scipy.stats.entropy(q, M)

def kl_divergence_discrete(before, data):
    keys = set(before.keys()).union(data.keys())
    print(len(keys))
    b_counts = [before.get(key,float_info.eps) for key in keys]
    c_counts = [data.get(key, float_info.eps) for key in keys]
    b_total = sum(b_counts)
    c_total = sum(c_counts)
    b_dist = [count / b_total for count in b_counts]
    c_dist = [count / c_total for count in c_counts]
    combined = sorted(zip(c_dist, b_dist), key=lambda x: x[0], reverse=True)
    sorted_a, sorted_b = zip(*combined)
    c_d = list(sorted_a)
    b_d = list(sorted_b)
    b = np.asarray(list(b_dist), dtype=np.float64)
    m = np.asarray(list(c_dist), dtype=np.float64)

    plt.figure()
    plt.bar(list(keys), b_d, label='Result Distribution', alpha=0.5)
    plt.bar(list(keys), c_d, label='Target Distribution', alpha=0.5)
    plt.legend()
    plt.title('Comparison of Two Distributions')
    plt.xlabel('Categories')
    plt.ylabel('Values')
    plt.show()

    return np.sum(m * np.log(m / b)), np.sum(b * np.log(b / m)), JS_divergence(b,m)
            
kl_mb, kl_bm, js_bm = kl_divergence_discrete(token_inter, top_100_prob)
print(kl_mb)
print(kl_bm)
print(js_bm)